# Process the Load Data for the NTP Heat Wave Grid Stress Events


In [1]:
# Start by importing the packages we need:
import os
import glob
import datetime

import pandas as pd
import numpy as np


## Set the Directory Structure

In [2]:
# Identify the data input and image output directories:
tell_data_input_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave_loads/data/TELL_Data/'
data_output_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave_loads/data/'


## Create a Function to Process the 2035 GridView Data Used in Scaling


In [73]:
def process_gridview_data(data_input_dir: str):
    # Check to see if the processed output file exist and if not then create it:
    if os.path.isfile((os.path.join(data_input_dir, 'Processed_Gridview_2035_Data.csv'))) == False:
       
       # Read in the raw data .csv file:
       gv_df = pd.read_csv((data_input_dir + 'wecc_load_2035.csv'))

       # Subset to just the annual total demand by BA:
       gv_df = gv_df[-3:-2]
       
       # Rename a few columns for consistency:
       gv_df.rename(columns={'Load_CIPB_2030_CEC.dat': 'Load_CIPB_2030.dat', 
                             'Load_CIPV_2030_CEC.dat': 'Load_CIPV_2030.dat',
                             'Load_CISC_2030_CEC.dat': 'Load_CISC_2030.dat',
                             'Load_CISD_2030_CEC.dat': 'Load_CISD_2030.dat'}, inplace=True)

       # Strip the unecessary bits from the column names:
       gv_df.columns = gv_df.columns.str.strip('_2030.dat')
       gv_df.columns = gv_df.columns.str.lstrip('Load_')  
       
       # Kluge the LDWP error:
       gv_df.rename(columns={'DWP': 'LDWP'}, inplace=True) 
        
       # Delete the index and last column:
       del gv_df["Index"], gv_df["Unnamed: 38"]
    
       # Compute the total loads for CISO, IPCO, NEVP, and PACE:
       gv_df['CISO'] = (gv_df['CIPB'] + gv_df['CIPV'] + gv_df['CISC'] + gv_df['CISD'] + gv_df['VEA']).round(2)
       gv_df['IPCO'] = (gv_df['IPFE'] + gv_df['IPMV'] + gv_df['IPTV']).round(2)
       gv_df['PACE'] = (gv_df['PAID'] + gv_df['PAUT'] + gv_df['PAWY']).round(2)
       gv_df['NEVP_Sum'] = (gv_df['NEVP'] + gv_df['SPPC']).round(2)
           
       # Rename a few columns for consistency:
       gv_df.rename(columns={'CIPB': 'CISO_CIPB', 'CIPV': 'CISO_CIPV', 'CISC': 'CISO_CISC', 'CISD': 'CISO_CISD', 'VEA': 'CISO_VEA',
                             'IPFE': 'IPCO_IPFE', 'IPMV': 'IPCO_IPMV', 'IPTV': 'IPCO_IPTV',
                             'NEVP': 'NEVP_NEVP', 'SPPC': 'NEVP_SPPC',
                             'PAID': 'PACE_PAID', 'PAUT': 'PACE_PAUT', 'PAWY': 'PACE_PAWY'}, inplace=True) 
       gv_df.rename(columns={'NEVP_Sum': 'NEVP'}, inplace=True) 
    
       # Squeeze the dataframe:
       gv_df = gv_df.squeeze().to_frame()
        
       # Rename the columns:
       gv_df.reset_index(inplace=True)
       gv_df = gv_df.rename(columns = {'index':'BA'})
       gv_df.rename(columns={gv_df.columns[1]: "Total_Load_MWh" }, inplace = True)
       
       # Sort the dataframe alphabetically by BA name:
       gv_df = gv_df.sort_values('BA')
       
       # Write out the dataframe to a .csv file:
       gv_df.to_csv((os.path.join(data_input_dir, 'Processed_Gridview_2035_Data.csv')), sep=',', index=False)
    
    else:
       # Read in the already processed output file:
       gv_df = pd.read_csv((os.path.join(data_input_dir, 'Processed_Gridview_2035_Data.csv')))       
    
    # Return the output dataframe:
    return gv_df


In [74]:
gv_df = process_gridview_data(data_input_dir = data_output_dir)

gv_df


,BA,Total_Load_MWh
0,AVA,1.700360e+07
1,AZPS,4.576623e+07
2,BANC,2.429027e+07
3,BPAT,6.571876e+07
4,CHPD,2.342334e+06
37,CISO,3.340924e+08
5,CISO_CIPB,5.583370e+07
6,CISO_CIPV,6.627299e+07
7,CISO_CISC,1.735778e+08
8,CISO_CISD,3.722391e+07


## Create a Function to Aggregate the Raw TELL MLP Output into a Single Dataframe:


In [75]:
def aggregate_mlp_output_files(tell_data_input_dir: str, year_to_process: str):
    
    # Create a list of all of the MLP output files in the "mlp_input_dir" and aggregate the files in that list:
    list_of_files = sorted(glob.glob(os.path.join(tell_data_input_dir, year_to_process, '*_mlp_output.csv')))

    # Loop over the list of MLP output files:
    for file in range(len(list_of_files)):

        # Read in the .csv file and replace missing values with nan:
        mlp_data = pd.read_csv(list_of_files[file]).replace(-9999, np.nan)

        # Rename the "Load" variable:
        mlp_data.rename(columns={'Load': 'Hourly_Load_MWh'}, inplace=True)

        # Replacing missing or negative loads with NaN:
        mlp_data.loc[~(mlp_data['Hourly_Load_MWh'] > 0), 'Hourly_Load_MWh'] = np.nan

        # Aggregate the output into a new dataframe:
        if file == 0:
            tell_df = mlp_data
        else:
            tell_df = pd.concat([tell_df, mlp_data])
    
    # Return the output dataframe:
    return tell_df


## Create a Function to Scale the TELL Output Based on the GridView 2035 Values:


In [81]:
def scale_tell_loads(data_input_dir: str, tell_data_input_dir: str, year_to_process: str):
    
    # Aggregate the TELL MLP files:
    tell_df = aggregate_mlp_output_files(tell_data_input_dir = tell_data_input_dir,
                                         year_to_process = year_to_process)
    
    # Read in the processed GridView file and rename a column for consistency:
    gv_df = pd.read_csv((os.path.join(data_input_dir, 'Processed_Gridview_2035_Data.csv')))
    gv_df.rename(columns={'Total_Load_MWh': 'GV_Total_Load_MWh'}, inplace=True) 
    
    # Merge the tell_df and gv_df dataframes based on common BA names:
    merged_df = tell_df.merge(gv_df, on=['BA'])
    
    # Sum the hourly TELL loads by BA into annual total loads:
    merged_df['TELL_Total_Load_MWh'] = merged_df.groupby('BA')['Hourly_Load_MWh'].transform('sum')
    
    # Compute the scaling factors that force the annual total loads to agree:
    merged_df['Scaling_Factor'] = merged_df['GV_Total_Load_MWh'] / merged_df['TELL_Total_Load_MWh']
    
    # Compute the scaled hourly loads:
    merged_df['Hourly_Load_MWh_Scaled'] = merged_df['Hourly_Load_MWh'] * merged_df['Scaling_Factor']
    
    # Compute the hours since the start of the year:
    merged_df['Hour'] = ((pd.to_datetime(merged_df['Time_UTC']) - datetime.datetime(int(year_to_process), 1, 1, 0, 0, 0)) / np.timedelta64(1, 'h') + 1).astype(int)
    
    # Only keep the columns that are needed:
    scaled_tell_df = merged_df[['Hour', 'BA', 'Hourly_Load_MWh_Scaled']].copy()
    
    # Drop the rows with missing values (i.e., there is not a corresponding GridView load):
    scaled_tell_df = scaled_tell_df.dropna(how = 'any')
    
    # Rename the load variable and round it to 5 decimals:
    scaled_tell_df.rename(columns={'Hourly_Load_MWh_Scaled': 'Load_MWh'}, inplace=True)
    scaled_tell_df['Load_MWh'] = scaled_tell_df['Load_MWh'].round(5)
    
    # Return the output dataframe:
    return scaled_tell_df


In [82]:
# Aggregate the TELL MLP files:
scaled_tell_df = scale_tell_loads(data_input_dir = data_output_dir,
                                  tell_data_input_dir = tell_data_input_dir, 
                                  year_to_process = '2055')

scaled_tell_df


,Hour,BA,Load_MWh
0,1,AVA,2581.04005
1,2,AVA,2659.71934
2,3,AVA,2728.42821
3,4,AVA,2785.34089
4,5,AVA,2719.74154
...,...,...,...
245275,8756,WAUW,126.68900
245276,8757,WAUW,125.70721
245277,8758,WAUW,124.20935
245278,8759,WAUW,123.05134


## Create a Function to Format the Output for Ingest to GridView:


In [83]:
def format_scaled_tell_loads(data_input_dir: str, tell_data_input_dir: str, year_to_process: str):
    
    # Aggregate the TELL MLP files:
    scaled_tell_df = scale_tell_loads(data_input_dir = data_output_dir,
                                      tell_data_input_dir = tell_data_input_dir, 
                                      year_to_process = year_to_process)
    
    # Read in the processed GridView file and rename a column for consistency:
    gv_df = pd.read_csv((os.path.join(data_input_dir, 'Processed_Gridview_2035_Data.csv')))
    
    # Compute the load fractions for the subregions:
    CIPB_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CIPB')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    CIPV_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CIPV')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    CISC_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CISC')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    CISD_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CISD')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    VEA_LF  = (gv_df.loc[(gv_df['BA'] == 'CISO_VEA' )]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    IPFE_LF = (gv_df.loc[(gv_df['BA'] == 'IPCO_IPFE')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['Total_Load_MWh'].values[0])
    IPMV_LF = (gv_df.loc[(gv_df['BA'] == 'IPCO_IPMV')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['Total_Load_MWh'].values[0])
    IPTV_LF = (gv_df.loc[(gv_df['BA'] == 'IPCO_IPTV')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['Total_Load_MWh'].values[0])
    NEVP_LF = (gv_df.loc[(gv_df['BA'] == 'NEVP_NEVP')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'NEVP')]['Total_Load_MWh'].values[0])
    SPPC_LF = (gv_df.loc[(gv_df['BA'] == 'NEVP_SPPC')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'NEVP')]['Total_Load_MWh'].values[0])
    PAID_LF = (gv_df.loc[(gv_df['BA'] == 'PACE_PAID')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['Total_Load_MWh'].values[0])
    PAUT_LF = (gv_df.loc[(gv_df['BA'] == 'PACE_PAUT')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['Total_Load_MWh'].values[0])
    PAWY_LF = (gv_df.loc[(gv_df['BA'] == 'PACE_PAWY')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['Total_Load_MWh'].values[0])
    
    # Reshape the dataframe and drop the indexes:
    output_df = scaled_tell_df.pivot(index = 'Hour', columns = 'BA', values = 'Load_MWh')
    output_df = output_df.reset_index(drop=False)
    
    # Add back in the text to the column headers:
    output_df = output_df.add_suffix('_2030.dat')
    output_df = output_df.add_prefix('Load_')
    
    # Rename the time variable:
    output_df.rename(columns={'Load_Hour_2030.dat': 'Index'}, inplace=True)
    
    # Compute the loads for the subregions:
    output_df['Load_CIPB_2030_CEC.dat'] = output_df['Load_CISO_2030.dat'] * CIPB_LF
    output_df['Load_CIPV_2030_CEC.dat'] = output_df['Load_CISO_2030.dat'] * CIPV_LF
    output_df['Load_CISC_2030_CEC.dat'] = output_df['Load_CISO_2030.dat'] * CISC_LF
    output_df['Load_CISD_2030_CEC.dat'] = output_df['Load_CISO_2030.dat'] * CISD_LF
    output_df['Load_VEA_2030.dat'] = output_df['Load_CISO_2030.dat'] * VEA_LF
    output_df['Load_IPFE_2030.dat'] = output_df['Load_IPCO_2030.dat'] * IPFE_LF
    output_df['Load_IPMV_2030.dat'] = output_df['Load_IPCO_2030.dat'] * IPMV_LF
    output_df['Load_IPTV_2030.dat'] = output_df['Load_IPCO_2030.dat'] * IPTV_LF
    output_df['Load_NEVP_Temp_2030.dat'] = output_df['Load_NEVP_2030.dat'] * NEVP_LF
    output_df['Load_SPPC_2030.dat'] = output_df['Load_NEVP_2030.dat'] * SPPC_LF
    output_df['Load_PAID_2030.dat'] = output_df['Load_PACE_2030.dat'] * PAID_LF
    output_df['Load_PAUT_2030.dat'] = output_df['Load_PACE_2030.dat'] * PAUT_LF
    output_df['Load_PAWY_2030.dat'] = output_df['Load_PACE_2030.dat'] * PAWY_LF
    
    # Drop the un-needed columns and clean up the NEVP naming:
    del output_df['Load_NEVP_2030.dat'], output_df['Load_CISO_2030.dat'], output_df['Load_IPCO_2030.dat'], output_df['Load_PACE_2030.dat']
    output_df.rename(columns={'Load_NEVP_Temp_2030.dat': 'Load_NEVP_2030.dat'}, inplace=True)
    
    # Sort the data by column name then make the Index column column one:
    output_df.rename(columns={'Index': 'AA'}, inplace=True)
    output_df = output_df.sort_index(axis = 1)
    output_df.rename(columns={'AA': 'Index'}, inplace=True)
    
    # Add in a blank row and fill it with the year placeholder:
    output_df.loc[-0.5] = 0
    output_df = output_df.sort_index().reset_index(drop=True)
    output_df.iloc[0, :] = '2030'
    output_df.at[0, 'Index'] = 'Year'
    
    # Set the output filenames:
    if year_to_process == '2055':
       output_filename = 'TELL_Loads_2035_Based_on_2015_Weather.csv'
    if year_to_process == '2058':
       output_filename = 'TELL_Loads_2035_Based_on_2018_Weather.csv'
    
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(data_input_dir, output_filename)), sep=',', index=False)
    
    # Return the output dataframe:
    return output_df


In [85]:
output_df = format_scaled_tell_loads(data_input_dir = data_output_dir,
                                     tell_data_input_dir = tell_data_input_dir,
                                     year_to_process = '2058')

output_df


BA,Index,Load_AVA_2030.dat,Load_AZPS_2030.dat,Load_BANC_2030.dat,Load_BPAT_2030.dat,Load_CHPD_2030.dat,Load_CIPB_2030_CEC.dat,Load_CIPV_2030_CEC.dat,Load_CISC_2030_CEC.dat,Load_CISD_2030_CEC.dat,...,Load_SCL_2030.dat,Load_SPPC_2030.dat,Load_SRP_2030.dat,Load_TEPC_2030.dat,Load_TIDC_2030.dat,Load_TPWR_2030.dat,Load_VEA_2030.dat,Load_WACM_2030.dat,Load_WALC_2030.dat,Load_WAUW_2030.dat
0,Year,2030,2030,2030,2030,2030,2030,2030,2030,2030,...,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030
1,1,2302.70353,3707.20348,2425.96068,8101.93393,397.79707,5446.402956,6464.722863,16931.970402,3631.076002,...,1489.35486,1272.998542,3972.06103,2270.11045,282.44503,765.5617,115.491417,4499.3142,1195.36872,168.30887
2,2,2383.40642,4015.76313,2590.61967,8580.28649,405.17337,5725.391113,6795.873739,17799.298662,3817.075313,...,1553.28507,1359.579199,4294.30746,2261.94637,321.97038,805.76323,121.407384,4549.50741,1228.15096,167.98663
3,3,2454.25117,4154.88147,2626.07895,8856.93789,399.05774,5904.029432,7007.912259,18354.655795,3936.172141,...,1600.96204,1396.469164,4422.17284,2213.75741,325.28449,824.32388,125.195424,4600.71859,1203.28265,167.72636
4,4,2519.82494,4264.82293,2728.19098,9102.19349,396.01334,6083.648254,7221.114609,18913.061151,4055.922662,...,1647.46002,1426.431504,4530.83024,2182.40525,330.97501,839.46456,129.004255,4546.12727,1194.45341,166.00361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,8756,2583.43278,4024.13094,2490.96046,9220.34908,385.1903,5640.694827,6695.341696,17535.99185,3760.608935,...,1667.19008,1353.625544,4459.99394,1300.03229,357.11544,821.43616,119.611392,4454.44569,1177.09065,143.22366
8757,8757,2553.91532,3996.66629,2470.72468,9079.2606,370.85346,5634.29853,6687.749476,17516.106815,3756.344571,...,1644.59675,1339.579895,4473.66026,1316.50391,356.75576,811.6519,119.475758,4451.65283,1174.54186,142.39327
8758,8758,2532.80293,3969.40876,2457.60742,8982.12382,360.23159,5662.256727,6720.935049,17603.024249,3774.984092,...,1634.9603,1324.120895,4482.83376,1386.56373,357.6421,805.54806,120.068614,4461.76716,1174.87982,141.51331
8759,8759,2529.48364,3940.10761,2442.13064,8895.24365,353.49904,5706.407065,6773.340224,17740.280384,3804.418791,...,1631.05038,1310.855153,4469.54315,1531.8218,358.25868,802.38856,121.004825,4455.63331,1181.89252,140.44743
